In [1]:
#!pip install pandas_datareader
#!pip install requests_cache
#!pip install yfinance

In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web 
import matplotlib.pyplot as plt
from IPython.display import display
import datetime
import requests_cache 
import plotly.graph_objects as go

### Заданные по условию параметры

In [3]:
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2019, 12, 31)
money = 100000
stop_loss = 0.07
short_window = 30
long_window = 90

### Выбор компании для анализа из MOEX

In [4]:
symbol=['SBER']

df = web.DataReader(symbol, 'moex', start=start, end=end)
df.head()

C:\Users\a7610\AppData\Local\Temp\ipykernel_10868\2158329127.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = web.DataReader(symbol, 'moex', start=start, end=end)


,ACCINT,ADMITTEDQUOTE,ADMITTEDVALUE,BEICLOSE,BID,BOARDID,BOARDNAME,BUYBACKDATE,CBRCLOSE,CLOSE,...,TRENDWAPPR,TYPE,VALUE,VOLUME,WAPRICE,WAVAL,YIELDATWAP,YIELDCLOSE,YIELDLASTCOUPON,YIELDTOOFFER
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2015-01-05,NaN,55.80,3.522353e+09,NaN,NaN,TQBR,NaN,NaN,NaN,56.37,...,NaN,NaN,3.528067e+09,63231040.0,55.80,NaN,NaN,NaN,NaN,NaN
2015-01-06,NaN,57.48,4.038267e+09,NaN,NaN,TQBR,NaN,NaN,NaN,58.28,...,NaN,NaN,4.039309e+09,70269700.0,57.48,NaN,NaN,NaN,NaN,NaN
2015-01-08,NaN,63.59,1.192409e+10,NaN,NaN,TQBR,NaN,NaN,NaN,65.70,...,NaN,NaN,1.192895e+10,187598890.0,63.59,NaN,NaN,NaN,NaN,NaN
2015-01-09,NaN,63.93,6.872282e+09,NaN,NaN,TQBR,NaN,NaN,NaN,63.10,...,NaN,NaN,6.875920e+09,107559890.0,63.93,NaN,NaN,NaN,NaN,NaN
2015-01-12,NaN,62.88,6.246868e+09,NaN,NaN,TQBR,NaN,NaN,NaN,62.90,...,NaN,NaN,6.252326e+09,99435900.0,62.88,NaN,NaN,NaN,NaN,NaN


### Подготовка данных по типу OHLC

In [5]:
ohlc=df[['OPEN','HIGH','LOW','CLOSE']]
ohlc.columns=['Open', 'High', 'Low', 'Close']
ohlc.head()

,Open,High,Low,Close
TRADEDATE,,,,
2015-01-05,54.02,56.77,53.58,56.37
2015-01-06,56.00,58.45,55.83,58.28
2015-01-08,59.00,67.77,58.61,65.70
2015-01-09,65.70,66.88,62.25,63.10
2015-01-12,62.20,63.84,61.66,62.90


### Расчет скользящих

In [6]:
short_rolling = ohlc['Open'].rolling(window=short_window, min_periods=0).mean()
long_rolling = ohlc['Open'].rolling(window=long_window, min_periods=0).mean()

### Подготовка данных для расчета моментов покупки на основании скользящих
##### без учета стоп-лоссов: 1 - акции куплены, 0 - вся позиция в кэше

In [7]:
when_in_cash = pd.DataFrame(index=ohlc.index)
when_in_cash['Action'] = 0
when_in_cash['Action'] = np.where(short_rolling > long_rolling, 1.0, 0.0)  
# сдвиг на день вперед, чтобы осуществлять покупки и продажи на следующий день после принятия решения
when_in_cash = when_in_cash.shift(1) 

#### Определение дней покупки

In [8]:
buy_sell = when_in_cash['Action'].diff()
buy_sell = pd.DataFrame(buy_sell)

In [9]:
buy_sig = buy_sell[(buy_sell['Action']==1)]
buy_sig = buy_sig.merge(ohlc, on='TRADEDATE', how='left')
buy_sig['Color'] = 'Green'
buy_sig

,Action,Open,High,Low,Close,Color
TRADEDATE,,,,,,
2015-02-18,1.0,73.38,76.84,72.83,76.40,Green
2015-07-23,1.0,74.61,74.69,72.51,72.70,Green
2015-09-11,1.0,74.28,74.42,73.69,74.10,Green
2016-03-04,1.0,107.70,109.02,106.70,108.99,Green
2017-06-05,1.0,157.10,158.89,154.43,154.70,Green
2017-08-09,1.0,174.80,175.20,172.80,173.40,Green
2018-11-27,1.0,189.98,192.45,188.11,191.03,Green
2019-01-29,1.0,209.71,214.49,206.91,213.61,Green
2019-10-24,1.0,239.63,242.78,239.06,242.78,Green


### Расчет стоп-лоссов для периодов, когда куплены акции

In [10]:
ohlc_with_signals = ohlc.copy()
buy_sig['Stop-loss'] = np.where(buy_sig['Action']==1, buy_sig['Open']*(1-stop_loss), 0)
ohlc_with_signals['Stop-loss'] = buy_sig['Stop-loss']
ohlc_with_signals['Stop-loss'] = ohlc_with_signals['Stop-loss'].fillna(method='ffill')
ohlc_with_signals['Stop-loss'] = np.where(when_in_cash['Action'] != 1, 0, ohlc_with_signals['Stop-loss'])
ohlc_with_signals.head()

,Open,High,Low,Close,Stop-loss
TRADEDATE,,,,,
2015-01-05,54.02,56.77,53.58,56.37,0.0
2015-01-06,56.00,58.45,55.83,58.28,0.0
2015-01-08,59.00,67.77,58.61,65.70,0.0
2015-01-09,65.70,66.88,62.25,63.10,0.0
2015-01-12,62.20,63.84,61.66,62.90,0.0


### Подготовка возможных сигналов к продаже

In [11]:
when_in_cash['Action'] = np.where(ohlc_with_signals['Stop-loss'] >= ohlc_with_signals['Low'], 0, when_in_cash['Action'])
buy_sell_with_stop = when_in_cash['Action'].diff()
buy_sell_with_stop = pd.DataFrame(buy_sell_with_stop)

In [12]:
sell_sig = buy_sell_with_stop[(buy_sell_with_stop['Action']==-1)]
sell_sig = sell_sig.merge(ohlc_with_signals, on='TRADEDATE', how='left')
sell_sig['Color'] = np.where(sell_sig['Stop-loss'] >= sell_sig['Low'], 'Black', 'Red')
sell_sig

,Action,Open,High,Low,Close,Stop-loss,Color
TRADEDATE,,,,,,,
2015-03-13,-1.0,71.00,71.30,67.81,67.87,68.2434,Black
2015-07-06,-1.0,69.93,70.59,68.08,68.45,68.2434,Black
2015-07-27,-1.0,70.40,70.45,67.35,68.48,69.3873,Black
2016-02-11,-1.0,95.71,96.10,93.05,94.20,0.0000,Red
2017-03-17,-1.0,158.55,162.85,158.17,161.15,0.0000,Red
2017-06-13,-1.0,145.35,147.30,145.00,146.00,146.1030,Black
2018-04-20,-1.0,214.00,216.27,212.13,215.36,0.0000,Red
2019-01-10,-1.0,196.25,197.64,195.03,196.75,0.0000,Red
2019-08-13,-1.0,221.31,225.75,220.07,223.18,0.0000,Red


#### Подтверждение, что стоп-лосс срабатывает в тот же день. На примере Сбера: взята дата за день до первого стоп-лосса - минимальная цена выше стоп-лосса.

In [13]:
ohlc_with_signals.loc['2015-03-12']

Open         71.1200
High         71.5900
Low          70.0200
Close        70.7900
Stop-loss    68.2434
Name: 2015-03-12 00:00:00, dtype: float64

### Соединение всех сигналов воедино

In [14]:
all_signals = pd.concat([buy_sig, sell_sig]).sort_index()
all_signals 

,Action,Open,High,Low,Close,Color,Stop-loss
TRADEDATE,,,,,,,
2015-02-18,1.0,73.38,76.84,72.83,76.40,Green,68.2434
2015-03-13,-1.0,71.00,71.30,67.81,67.87,Black,68.2434
2015-07-06,-1.0,69.93,70.59,68.08,68.45,Black,68.2434
2015-07-23,1.0,74.61,74.69,72.51,72.70,Green,69.3873
2015-07-27,-1.0,70.40,70.45,67.35,68.48,Black,69.3873
2015-09-11,1.0,74.28,74.42,73.69,74.10,Green,69.0804
2016-02-11,-1.0,95.71,96.10,93.05,94.20,Red,0.0000
2016-03-04,1.0,107.70,109.02,106.70,108.99,Green,100.1610
2017-03-17,-1.0,158.55,162.85,158.17,161.15,Red,0.0000


###  Устранение лишних стоп-лоссов

In [15]:
correct_signals = pd.DataFrame()
correct_signals = pd.concat([correct_signals, pd.DataFrame(all_signals.iloc[0])], axis=1)
for i in range (len(all_signals) - 1):
    if all_signals.Action[i] != all_signals.Action[i+1]:
        correct_signals = pd.concat([correct_signals, pd.DataFrame(all_signals.iloc[i+1])], axis=1)

signals = correct_signals.T
sell_sig_stop = signals[signals['Color']=='Black']
sell_sig_without_stop = signals[signals['Color']=='Red']
signals 

,Action,Open,High,Low,Close,Color,Stop-loss
2015-02-18,1.0,73.38,76.84,72.83,76.4,Green,68.2434
2015-03-13,-1.0,71.0,71.3,67.81,67.87,Black,68.2434
2015-07-23,1.0,74.61,74.69,72.51,72.7,Green,69.3873
2015-07-27,-1.0,70.4,70.45,67.35,68.48,Black,69.3873
2015-09-11,1.0,74.28,74.42,73.69,74.1,Green,69.0804
2016-02-11,-1.0,95.71,96.1,93.05,94.2,Red,0.0
2016-03-04,1.0,107.7,109.02,106.7,108.99,Green,100.161
2017-03-17,-1.0,158.55,162.85,158.17,161.15,Red,0.0
2017-06-05,1.0,157.1,158.89,154.43,154.7,Green,146.103
2017-06-13,-1.0,145.35,147.3,145.0,146.0,Black,146.103


### Формирование данных по позициям на основании сигналов
#### 1 - акции куплены, -1 - позиция в кэше

In [16]:
position = pd.DataFrame()
position.index = ohlc.index
position['Action'] = signals['Action']
position['Action'] = position['Action'].fillna(method='ffill')
position['Action'] = position['Action'].fillna(-1) # заполнение интервала до первой покупки

### Визуализация результатов
#### На нижнем графике "Shares" - позиция в акциях, "Cash" - позиция в кэше, rangeselector выбирает интервал от правой границы влево.

In [17]:
trace1 = go.Candlestick(x=ohlc.index, open=ohlc['Open'], high=ohlc['High'],
                   low=ohlc['Low'], close=ohlc['Close'],
                       yaxis = "y1", showlegend=False)
trace2 = go.Scatter(x=ohlc.index, y=short_rolling, name='Short_rolling', line=dict(color='Blue'),
                       yaxis = "y1")
trace3 = go.Scatter(x=ohlc.index, y=long_rolling, name='Long_rolling',
                       yaxis = "y1")
trace4 = go.Scatter(
        mode='markers',
        x=buy_sig.index,
        y=(buy_sig.Low-buy_sig.Low*0.05),
        name='Buy',
        yaxis = "y1",
        marker=dict(
            symbol='triangle-up',
            color='LightGreen',
            size=10,
            line=dict(color='Green', width=2)
        ))
trace5 = go.Scatter(
        mode='markers',
        x=sell_sig_stop.index,
        y=(sell_sig_stop.High+sell_sig_stop.High*0.05),
        name='Stop-loss',
        yaxis = "y1",
        marker=dict(
            symbol='triangle-down',
            color='Black',
            size=10,
            line=dict(color='Grey', width=2)
        ))
trace6 = go.Scatter(
        mode='markers',
        x=sell_sig_without_stop.index,
        y=(sell_sig_without_stop.High+sell_sig_without_stop.High*0.05),
        name='Sell',
        yaxis = "y1",
        marker=dict(
            symbol='triangle-down',
            color='Red',
            size=10,
            line=dict(color='Black', width=2)
        ))
trace7 = go.Scatter(x=position.index, y=position['Action'],
        yaxis = "y2", 
        showlegend=False)
layout = dict(
    height=1000,
    title=symbol[0],
    yaxis_title='Stock',
    yaxis1 = dict(title_text="Candlestick", domain= [0.35, 1]),
    yaxis2 = dict(title_text="Position", tickvals=[-1,1], ticktext =['Cash', 'Shares'], domain= [0.0, 0.25]),
    xaxis = dict(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    ),
         rangeslider=dict(
            visible = True
        ),
        type='date'
))

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]
    
fig = go.FigureWidget(data=data, layout=layout)

def zoom(layout, xrange):
    in_view = ohlc.loc[fig.layout.xaxis.range[0]:fig.layout.xaxis.range[1]]
    fig.layout.yaxis.range = [in_view.Low.min() - in_view.Low.min()*0.1, in_view.High.max() + in_view.High.max()*0.1]

fig.layout.on_change(zoom, 'xaxis.range') 
fig.update_layout(xaxis = dict(rangemode = 'normal')) #, bounds = [start, end]))
fig

FigureWidget({
    'data': [{'close': array([ 56.37,  58.28,  65.7 , ..., 248.24, 252.06, 254.75]),
          …

### Подготовка данных для итоговой таблицы
#### Сдвиги на день осуществляются, чтобы по датам покупок и продаж можно было добавить данные по сигналами к продаже и покупке за предыдущий день. 
#### Из описания задания мне оказалось не очень понятным, какие значения share_price подбирать для sig_buy и sig_sell. В качестве этих параметов были выбраны сигналы: значение короткой скользящей для sig_buy в день сигнала, значение длинной скользящей для sig_sell в день сигнала. Чтобы выбрать, например, цену закрытия: share_price = ohlc.Close.loc[date], где date = date_shift['Shift'].loc[idx]

In [18]:
short_rolling_shift = pd.DataFrame(short_rolling).shift(1)
long_rolling_shift = pd.DataFrame(long_rolling).shift(1)
date_shift = pd.DataFrame()
date_shift.index = ohlc.index
date_shift['Shift'] = ohlc.index
date_shift['Shift'] = date_shift['Shift'].shift(1) 

### Формирование итоговой таблицы

In [19]:
num_shares = 0
summary = pd.DataFrame()
signal = ()
cash = money
for idx, sign in signals.iterrows():
    if sign.Color == 'Green':
        signal = 'sig_buy'
        date = date_shift['Shift'].loc[idx]
        share_price = short_rolling_shift['Open'].loc[idx]
        num_shares = 0
        share_value = 0
        summary = pd.concat([summary, pd.Series({'date': date, 'signal': signal, 'num_shares': int(num_shares), 'share_price': share_price, 'share_value': share_value, 'cash': cash})], axis=1)
        
        signal = 'buy'
        num_shares = cash//sign.Open
        share_value = sign.Open*num_shares
        cash = (money%sign.Open if num_shares >0 else cash)
        count_of_shares = num_shares
        summary = pd.concat([summary, pd.Series({'date':idx, 'signal': signal, 'num_shares': int(num_shares), 'share_price': sign.Open, 'share_value': share_value, 'cash': cash})], axis=1)

    elif sign.Color == 'Red':
        signal = 'sig_sale'
        date = date_shift['Shift'].loc[idx]
        share_price = long_rolling_shift['Open'].loc[idx]
        share_value = num_shares * share_price
        summary = pd.concat([summary, pd.Series({'date': date, 'signal': signal, 'num_shares': int(num_shares), 'share_price': share_price, 'share_value': share_value, 'cash': cash})], axis=1)
        
        signal = 'sale'
        num_shares = count_of_shares 
        share_value = 0
        cash += sign.Open*num_shares
        summary = pd.concat([summary, pd.Series({'date':idx, 'signal': signal, 'num_shares': int(num_shares), 'share_price': sign.Open, 'share_value': share_value, 'cash': cash})], axis=1)
    elif sign.Color == 'Black':
        signal = 'stop-loss'
        num_shares = count_of_shares 
        share_value = 0
        cash += sign['Stop-loss']*num_shares
        summary = pd.concat([summary, pd.Series({'date':idx, 'signal': signal, 'num_shares': int(num_shares), 'share_price': sign['Stop-loss'], 'share_value': share_value, 'cash': cash})], axis=1)        
        
summary = summary.T.reset_index(drop=True)
summary

,date,signal,num_shares,share_price,share_value,cash
0,2015-02-17,sig_buy,0,62.889667,0,100000
1,2015-02-18,buy,1362,73.38,99943.56,56.44
2,2015-03-13,stop-loss,1362,68.2434,0,93003.9508
3,2015-07-22,sig_buy,0,71.765667,0,93003.9508
4,2015-07-23,buy,1246,74.61,92964.06,22.6
5,2015-07-27,stop-loss,1246,69.3873,0,86479.1758
6,2015-09-10,sig_buy,0,72.787333,0,86479.1758
7,2015-09-11,buy,1164,74.28,86461.92,19.12
8,2016-02-10,sig_sale,1164,94.364889,109840.730667,19.12
9,2016-02-11,sale,1164,95.71,0,111425.56


### Вывод итоговых данных на момент последней продажи

In [20]:
if summary['signal'].iloc[-1] == 'buy':
    profit = summary['cash'].iloc[-3]
else:
    profit = summary['cash'].iloc[-1]
print('Итоговая стоимость:', profit, '\nПроцент прироста/падения:', (profit-money)/money*100, '%')

Итоговая стоимость: 203561.93000000002 
Процент прироста/падения: 103.56193000000002 %
